# 合同会社 長目 セミナーノートブック

## 位置データ活用　経済センサスのデータを使ってみよう！

### やること
- 経済センサスのデータを使ってヘルシーな昼食で稼ぐ店の出店候補地を探す！

In [ ]:
# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal &> /dev/null
# Install rtree - Geopandas requirment
!apt install python3-rtree &> /dev/null
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git &> /dev/null
# Install descartes - Geopandas requirment
!pip install descartes &> /dev/null
!pip install plotly -U &> /dev/null

In [ ]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
import folium
import shapely

In [ ]:
position_data_path = '/content/drive/MyDrive/work/20211117_seminer/data/data/H26ca26.shp'
kiso_data_path = '/content/drive/MyDrive/work/20211117_seminer/data/data/kiso_data_kyoto.csv'

df = pd.read_csv(kiso_data_path, index_col=0) # 数値データ
data = gpd.read_file(position_data_path) # 位置データ

In [ ]:
# データの概観観察
df

,KEY_CODE,CITY_NAME,AZA_CODE,AZA_NAME,総数（Ａ～Ｓ全産業）,Ａ～Ｒ全産業（Ｓ公務を除く）,Ａ～Ｂ農林漁業,Ｃ～Ｓ非農林漁業,Ｃ～Ｒ非農林漁業（Ｓ公務を除く）,Ｃ鉱業、採石業、砂利採取業,Ｄ建設業,Ｅ製造業,Ｆ電気・ガス・熱供給・水道業,Ｇ情報通信業,Ｈ運輸業、郵便業,Ｉ卸売業、小売業,Ｊ金融業、保険業,Ｋ不動産業、物品賃貸業,Ｌ学術研究、専門・技術サービス業,Ｍ宿泊業、飲食サービス業,Ｎ生活関連サービス業、娯楽業,Ｏ教育、学習支援業,Ｐ医療、福祉,Ｑ複合サービス事業,Ｒサービス業（他に分類されないもの）,Ｓ公務（他に分類されるものを除く）,１～４人,５～９人,１０～１９人,２０～２９人,３０人以上,出向・派遣従業者のみ,総数（Ａ～Ｓ全産業）.1,男（Ａ～Ｓ全産業）,女（Ａ～Ｓ全産業）,Ａ～Ｒ全産業（Ｓ公務を除く）.1,男Ａ～Ｒ全産業（Ｓ公務を除く）,女Ａ～Ｒ全産業（Ｓ公務を除く）,Ａ～Ｂ農林漁業.1,Ｃ～Ｓ非農林漁業.1,Ｃ～Ｒ非農林漁業（Ｓ公務を除く）.1,Ｃ鉱業、採石業、砂利採取業.1,Ｄ建設業.1,Ｅ製造業.1,Ｆ電気・ガス・熱供給・水道業.1,Ｇ情報通信業.1,Ｈ運輸業、郵便業.1,Ｉ卸売業、小売業.1,Ｊ金融業、保険業.1,Ｋ不動産業、物品賃貸業.1,Ｌ学術研究、専門・技術サービス業.1,Ｍ宿泊業、飲食サービス業.1,Ｎ生活関連サービス業、娯楽業.1,Ｏ教育、学習支援業.1,Ｐ医療、福祉.1,Ｑ複合サービス事業.1,Ｒサービス業（他に分類されないもの）.1,Ｓ公務（他に分類されるものを除く）.1,１～４人.1,５～９人.1,１０～１９人.1,２０～２９人.1,３０人以上.1,民営事業所数,うち個人,うち法人,うち会社,会社以外の法人,民営従業者数,うち個人.1,うち法人.1,うち会社.1,会社以外の法人.1
0,2.610100e+14,京都市北区,1.000001e+08,出雲路神楽町,5,5,-,5,5,-,2,-,-,-,-,2,-,-,-,1,-,-,-,-,-,-,3,1,1,-,-,-,29,24,5,29,24,5,-,29,29,-,10,-,-,-,-,17,-,-,-,2,-,-,-,-,-,-,5,8,16,-,-,5,2,3,3,-,29,3,26,26,-
1,2.610100e+14,京都市北区,2.000002e+08,出雲路立テ本町,6,6,-,6,6,-,1,-,-,-,-,1,-,1,-,-,1,1,1,-,-,-,5,-,-,1,-,-,35,9,26,35,9,26,-,35,35,-,2,-,-,-,-,4,-,3,-,-,2,1,23,-,-,-,12,-,-,23,-,6,1,5,5,-,35,2,33,33,-
2,2.610100e+14,京都市北区,3.000003e+08,出雲路俵町,14,14,-,14,14,-,-,1,-,-,-,4,-,-,-,4,2,-,1,-,2,-,9,3,2,-,-,-,76,30,46,76,30,46,-,76,76,-,-,18,-,-,-,17,-,-,-,18,6,-,8,-,9,-,24,24,28,-,-,14,3,11,9,2,76,9,67,57,10
3,2.610100e+14,京都市北区,4.000004e+08,出雲路松ノ下町,5,5,-,5,5,-,-,-,-,-,-,-,-,1,-,1,-,-,1,-,2,-,4,1,-,-,-,-,13,6,7,13,6,7,-,13,13,-,-,-,-,-,-,-,-,1,-,5,-,-,3,-,4,-,8,5,-,-,-,5,2,3,2,1,13,4,9,6,3
4,2.610100e+14,京都市北区,5.000005e+08,大北山鏡石町,3,3,-,3,3,-,-,1,-,-,-,1,-,-,-,-,-,-,-,-,1,-,1,1,1,-,-,-,20,17,3,20,17,3,-,20,20,-,-,5,-,-,-,14,-,-,-,-,-,-,-,-,1,-,1,5,14,-,-,3,-,3,1,2,20,-,20,5,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7545,2.646500e+14,与謝郡与謝野町,2.400002e+09,字滝,39,39,3,36,36,-,7,10,-,-,-,8,-,-,-,3,1,2,2,1,2,-,24,9,4,2,-,-,191,88,103,191,88,103,58,133,133,-,33,21,-,-,-,29,-,-,-,11,1,15,14,5,4,-,43,54,48,46,-,32,22,10,9,1,135,54,81,79,2
7546,2.646500e+14,与謝郡与謝野町,2.500002e+09,字三河内,201,201,-,201,201,-,20,93,-,-,1,33,2,2,6,7,10,11,10,2,4,-,162,22,11,2,4,-,841,406,435,841,406,435,-,841,841,-,87,233,-,-,2,192,6,10,23,17,40,95,93,10,33,-,309,152,162,45,173,196,149,47,40,7,751,326,425,337,88
7547,2.646500e+14,与謝郡与謝野町,2.600003e+09,字弓木,108,108,-,108,108,-,13,51,-,-,3,12,1,3,4,2,10,2,4,-,3,-,77,20,5,4,2,-,574,355,219,574,355,219,-,574,574,-,74,268,-,-,6,46,2,7,7,33,44,25,47,-,15,-,144,134,64,92,140,107,66,40,33,7,566,151,391,336,55
7548,2.646500e+14,与謝郡与謝野町,2.700003e+09,字与謝,30,30,2,28,28,-,1,22,-,-,-,3,-,-,-,1,-,-,-,-,1,-,27,2,1,-,-,-,80,50,30,80,50,30,10,70,70,-,13,46,-,-,-,8,-,-,-,2,-,-,-,-,1,-,54,13,13,-,-,29,24,5,4,1,78,47,31,30,1


In [ ]:
# '-'がある場所を0に置き換え、数値データを数値型にする
df.iloc[:, 4:] = df.iloc[:, 4:].replace('-', '0').copy()
df.iloc[:, 4:] = df.iloc[:, 4:].astype(int)
# KEY_CODEもついでに数値型に置き換える
df['KEY_CODE'] = df['KEY_CODE'].astype(int)
cols = list()
for col in df.columns:
    cols.append(col.replace('.', '_'))

df.columns = cols

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7550 entries, 0 to 7549
Data columns (total 73 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   KEY_CODE              7550 non-null   int64  
 1   CITY_NAME             6470 non-null   object 
 2   AZA_CODE              6470 non-null   float64
 3   AZA_NAME              6470 non-null   object 
 4   総数（Ａ～Ｓ全産業）            7550 non-null   int64  
 5   Ａ～Ｒ全産業（Ｓ公務を除く）        7550 non-null   int64  
 6   Ａ～Ｂ農林漁業               7550 non-null   int64  
 7   Ｃ～Ｓ非農林漁業              7550 non-null   int64  
 8   Ｃ～Ｒ非農林漁業（Ｓ公務を除く）      7550 non-null   int64  
 9   Ｃ鉱業、採石業、砂利採取業         7550 non-null   int64  
 10  Ｄ建設業                  7550 non-null   int64  
 11  Ｅ製造業                  7550 non-null   int64  
 12  Ｆ電気・ガス・熱供給・水道業        7550 non-null   int64  
 13  Ｇ情報通信業                7550 non-null   int64  
 14  Ｈ運輸業、郵便業              7550 non-null   int64  
 15  Ｉ卸売業、小売業             

## 条件にマッチする地域を探す

- 働いている人が多い
    - 32  総数（Ａ～Ｓ全産業）.1
- 女性が多い
    - 34  女（Ａ～Ｓ全産業）
- ランチのライバルが少なそう
    - 51  Ｍ宿泊業、飲食サービス業.1


In [ ]:
df['women_ratio'] = df['女（Ａ～Ｓ全産業）'] / df['総数（Ａ～Ｓ全産業）_1']

In [ ]:
df[['総数（Ａ～Ｓ全産業）_1', 'women_ratio', 'Ｍ宿泊業、飲食サービス業_1', '３０人以上']].describe()

,総数（Ａ～Ｓ全産業）_1,women_ratio,Ｍ宿泊業、飲食サービス業_1,３０人以上
count,7550.000000,6469.000000,7550.000000,7550.000000
mean,164.265166,0.454645,17.232318,0.928609
std,499.979672,0.181744,86.414211,3.047435
min,0.000000,0.000000,0.000000,0.000000
25%,9.000000,0.341317,0.000000,0.000000
50%,46.000000,0.465347,0.000000,0.000000
75%,146.000000,0.571429,10.000000,1.000000
max,19782.000000,1.000000,5523.000000,139.000000


In [ ]:
px.histogram(df[df["Ｍ宿泊業、飲食サービス業_1"] <= 10]['Ｍ宿泊業、飲食サービス業_1'])

In [ ]:
df_total = df[df['女（Ａ～Ｓ全産業）'] > 146 ]['女（Ａ～Ｓ全産業）']
px.histogram(df_total, title=f'データ数: {len(df_total)}')

In [ ]:
df_area = df[(df["Ｍ宿泊業、飲食サービス業_1"] <= 10) & (df["総数（Ａ～Ｓ全産業）_1"] > 146) & (df["women_ratio"] > 0.6)]
len(df_area)

112

In [ ]:
df_area[df_area["３０人以上"] > 0]

,KEY_CODE,CITY_NAME,AZA_CODE,AZA_NAME,総数（Ａ～Ｓ全産業）,Ａ～Ｒ全産業（Ｓ公務を除く）,Ａ～Ｂ農林漁業,Ｃ～Ｓ非農林漁業,Ｃ～Ｒ非農林漁業（Ｓ公務を除く）,Ｃ鉱業、採石業、砂利採取業,Ｄ建設業,Ｅ製造業,Ｆ電気・ガス・熱供給・水道業,Ｇ情報通信業,Ｈ運輸業、郵便業,Ｉ卸売業、小売業,Ｊ金融業、保険業,Ｋ不動産業、物品賃貸業,Ｌ学術研究、専門・技術サービス業,Ｍ宿泊業、飲食サービス業,Ｎ生活関連サービス業、娯楽業,Ｏ教育、学習支援業,Ｐ医療、福祉,Ｑ複合サービス事業,Ｒサービス業（他に分類されないもの）,Ｓ公務（他に分類されるものを除く）,１～４人,５～９人,１０～１９人,２０～２９人,３０人以上,出向・派遣従業者のみ,総数（Ａ～Ｓ全産業）_1,男（Ａ～Ｓ全産業）,女（Ａ～Ｓ全産業）,Ａ～Ｒ全産業（Ｓ公務を除く）_1,男Ａ～Ｒ全産業（Ｓ公務を除く）,女Ａ～Ｒ全産業（Ｓ公務を除く）,Ａ～Ｂ農林漁業_1,Ｃ～Ｓ非農林漁業_1,Ｃ～Ｒ非農林漁業（Ｓ公務を除く）_1,Ｃ鉱業、採石業、砂利採取業_1,Ｄ建設業_1,Ｅ製造業_1,Ｆ電気・ガス・熱供給・水道業_1,Ｇ情報通信業_1,Ｈ運輸業、郵便業_1,Ｉ卸売業、小売業_1,Ｊ金融業、保険業_1,Ｋ不動産業、物品賃貸業_1,Ｌ学術研究、専門・技術サービス業_1,Ｍ宿泊業、飲食サービス業_1,Ｎ生活関連サービス業、娯楽業_1,Ｏ教育、学習支援業_1,Ｐ医療、福祉_1,Ｑ複合サービス事業_1,Ｒサービス業（他に分類されないもの）_1,Ｓ公務（他に分類されるものを除く）_1,１～４人_1,５～９人_1,１０～１９人_1,２０～２９人_1,３０人以上_1,民営事業所数,うち個人,うち法人,うち会社,会社以外の法人,民営従業者数,うち個人_1,うち法人_1,うち会社_1,会社以外の法人_1,women_ratio
7,261010000000008,京都市北区,8.000008e+08,大北山長谷町,9,9,0,9,9,0,1,2,0,0,0,1,0,0,0,0,1,0,4,0,0,0,4,1,1,0,2,1,204,74,130,204,74,130,0,204,204,0,2,7,0,0,0,2,0,0,0,0,35,0,158,0,0,0,8,6,12,0,178,9,2,7,4,3,204,5,199,44,155,0.637255
39,261010000000040,京都市北区,4.000004e+09,大宮南山ノ前町,18,18,0,18,18,0,0,3,0,0,0,5,1,1,0,3,2,1,2,0,0,0,10,5,1,0,2,0,152,50,102,152,50,102,0,152,152,0,0,14,0,0,0,37,4,1,0,7,6,5,78,0,0,0,25,31,18,0,78,18,10,8,6,2,152,44,108,30,78,0.671053
132,261010000000133,京都市北区,1.330001e+10,衣笠衣笠山町,6,6,0,6,6,0,0,0,0,0,0,0,0,0,0,0,0,4,2,0,0,0,1,0,1,1,3,0,164,35,129,164,35,129,0,164,164,0,0,0,0,0,0,0,0,0,0,0,0,94,70,0,0,0,3,0,15,25,121,5,0,5,0,5,118,0,118,0,118,0.786585
207,261010000000209,京都市北区,2.090002e+10,紫竹西南町,34,34,0,34,34,0,2,8,0,0,0,4,1,3,4,2,2,2,4,0,2,0,28,2,2,0,2,0,179,51,128,179,51,128,0,179,179,0,5,30,0,0,0,9,2,7,7,10,3,2,99,0,5,0,58,13,25,0,83,34,20,13,10,3,179,42,134,49,85,0.715084
243,261010000000245,京都市北区,2.450002e+10,鷹峯土天井町,12,12,0,12,12,0,1,1,0,0,0,1,1,2,1,1,0,0,4,0,0,0,5,2,1,2,2,0,295,81,214,295,81,214,0,295,295,0,5,9,0,0,0,4,3,3,4,10,0,0,257,0,0,0,14,14,10,49,208,12,2,10,6,4,295,6,289,32,257,0.725424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7284,262140000000049,木津川市,1.360001e+10,相楽台９丁目,9,9,0,9,9,0,0,0,0,0,0,1,0,2,0,0,1,1,3,0,1,0,5,0,1,1,2,0,162,51,111,162,51,111,0,162,162,0,0,0,0,0,0,40,0,5,0,0,3,28,73,0,13,0,14,0,13,28,107,9,2,7,6,1,162,5,157,153,4,0.685185
7292,262140000000057,木津川市,1.490002e+10,加茂町駅東４丁目,7,7,0,7,7,0,1,1,0,0,0,1,0,0,0,0,0,2,2,0,0,0,3,0,1,1,2,0,237,65,172,237,65,172,0,237,237,0,2,25,0,0,0,1,0,0,0,0,0,19,190,0,0,0,7,0,15,25,190,7,1,5,3,2,237,1,221,31,190,0.725738
7408,263660000000008,相楽郡精華町,8.700009e+09,大字山田,38,38,0,38,38,0,3,2,0,0,0,7,1,4,2,1,5,4,4,1,4,0,26,8,0,1,3,0,556,200,356,556,200,356,0,556,556,0,15,8,0,0,0,31,7,6,8,2,8,221,38,6,206,0,66,53,0,23,414,37,17,20,17,3,370,46,324,287,37,0.640288
7422,263660000000022,相楽郡精華町,1.010001e+10,精華台１丁目,14,14,0,14,14,0,0,0,0,0,0,6,0,0,1,0,0,5,2,0,0,0,6,2,2,3,1,0,176,69,107,176,69,107,0,176,176,0,0,0,0,0,0,69,0,0,1,0,0,97,9,0,0,0,13,17,25,74,47,13,4,9,9,0,129,11,118,118,0,0.607955


## 数値データのみだと
- 人口密度が分からない
- 位置データが欲しくなる

## 位置データとマージしたデータを作成する

In [ ]:
data

,KEY_CODE,PREF,CITY,S_AREA,PREF_NAME,CITY_NAME,S_NAME,KIGO_E,AREA,PERIMETER,KEN,KEN_NAME,CSS_NAME,MOJI,AZA_CODE,AREA_MAX_F,KIGO_D,N_KEN,N_CITY,KIGO_I,JIGYOSHO,JUGYOSHA,X_CODE,Y_CODE,KEYCODE1,geometry
0,261010000000001,26,101,000100000100,京都府,京都市北区,出雲路神楽町,None,4.843208e+04,1040.540,26,京都府,京都市北区,出雲路神楽町,000100000100,M,None,None,None,None,5,29,135.76830,35.03372,1010000000001,"POLYGON ((135.76674 35.03474, 135.76688 35.034..."
1,261010000000002,26,101,000200000200,京都府,京都市北区,出雲路立テ本町,None,6.134630e+04,1512.663,26,京都府,京都市北区,出雲路立テ本町,000200000200,M,None,None,None,None,6,35,135.76441,35.03951,1010000000002,"POLYGON ((135.76281 35.04064, 135.76292 35.040..."
2,261010000000003,26,101,000300000300,京都府,京都市北区,出雲路俵町,None,6.526348e+04,1123.113,26,京都府,京都市北区,出雲路俵町,000300000300,M,None,None,None,None,14,76,135.76633,35.03655,1010000000003,"POLYGON ((135.76633 35.03832, 135.76657 35.037..."
3,261010000000004,26,101,000400000400,京都府,京都市北区,出雲路松ノ下町,None,5.279761e+04,1096.054,26,京都府,京都市北区,出雲路松ノ下町,000400000400,M,None,None,None,None,5,13,135.76300,35.04192,1010000000004,"POLYGON ((135.76154 35.04247, 135.76162 35.042..."
4,261010000000005,26,101,000500000500,京都府,京都市北区,大北山鏡石町,None,2.359074e+05,3045.946,26,京都府,京都市北区,大北山鏡石町,000500000500,M,None,None,None,None,3,20,135.73145,35.04665,1010000000005,"POLYGON ((135.73338 35.04831, 135.73332 35.047..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8284,264650000000016,26,465,002400002400,京都府,与謝郡与謝野町,字滝,None,1.557659e+07,24965.777,26,京都府,与謝郡与謝野町,字滝,002400002400,M,None,None,None,None,39,191,135.05751,35.46886,4650000000016,"POLYGON ((135.07700 35.48826, 135.07701 35.488..."
8285,264650000000017,26,465,002500002500,京都府,与謝郡与謝野町,字三河内,None,3.683443e+06,10826.044,26,京都府,与謝郡与謝野町,字三河内,002500002500,M,None,None,None,None,201,841,135.08837,35.51785,4650000000017,"POLYGON ((135.09067 35.52668, 135.09071 35.526..."
8286,264650000000018,26,465,002600002600,京都府,与謝郡与謝野町,字弓木,None,2.863092e+06,11378.146,26,京都府,与謝郡与謝野町,字弓木,002600002600,M,None,None,None,None,108,574,135.13991,35.55921,4650000000018,"POLYGON ((135.13857 35.56881, 135.13881 35.568..."
8287,264650000000019,26,465,002700002700,京都府,与謝郡与謝野町,字与謝,None,1.147165e+07,14232.546,26,京都府,与謝郡与謝野町,字与謝,002700002700,M,None,None,None,None,30,80,135.08430,35.45826,4650000000019,"POLYGON ((135.09537 35.47244, 135.09540 35.472..."


In [ ]:
data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 8289 entries, 0 to 8288
Data columns (total 26 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   KEY_CODE    8288 non-null   object  
 1   PREF        8289 non-null   object  
 2   CITY        8289 non-null   object  
 3   S_AREA      8288 non-null   object  
 4   PREF_NAME   8288 non-null   object  
 5   CITY_NAME   8288 non-null   object  
 6   S_NAME      8275 non-null   object  
 7   KIGO_E      1106 non-null   object  
 8   AREA        8289 non-null   float64 
 9   PERIMETER   8289 non-null   float64 
 10  KEN         8289 non-null   object  
 11  KEN_NAME    8288 non-null   object  
 12  CSS_NAME    8288 non-null   object  
 13  MOJI        8275 non-null   object  
 14  AZA_CODE    8288 non-null   object  
 15  AREA_MAX_F  7582 non-null   object  
 16  KIGO_D      21 non-null     object  
 17  N_KEN       21 non-null     object  
 18  N_CITY      21 non-null     object  
 19

In [ ]:
# マージのためにKEY_CODEのデータ型を変える必要がある

data.merge(df, on='KEY_CODE')


ValueError: ignored

In [ ]:
# None が含まれるため簡単にint型に変更できない

data['KEY_CODE'].astype(int)

TypeError: ignored

In [ ]:
data

,KEY_CODE,PREF,CITY,S_AREA,PREF_NAME,CITY_NAME,S_NAME,KIGO_E,AREA,PERIMETER,KEN,KEN_NAME,CSS_NAME,MOJI,AZA_CODE,AREA_MAX_F,KIGO_D,N_KEN,N_CITY,KIGO_I,JIGYOSHO,JUGYOSHA,X_CODE,Y_CODE,KEYCODE1,geometry
0,261010000000001,26,101,000100000100,京都府,京都市北区,出雲路神楽町,None,4.843208e+04,1040.540,26,京都府,京都市北区,出雲路神楽町,000100000100,M,None,None,None,None,5,29,135.76830,35.03372,1010000000001,"POLYGON ((135.76674 35.03474, 135.76688 35.034..."
1,261010000000002,26,101,000200000200,京都府,京都市北区,出雲路立テ本町,None,6.134630e+04,1512.663,26,京都府,京都市北区,出雲路立テ本町,000200000200,M,None,None,None,None,6,35,135.76441,35.03951,1010000000002,"POLYGON ((135.76281 35.04064, 135.76292 35.040..."
2,261010000000003,26,101,000300000300,京都府,京都市北区,出雲路俵町,None,6.526348e+04,1123.113,26,京都府,京都市北区,出雲路俵町,000300000300,M,None,None,None,None,14,76,135.76633,35.03655,1010000000003,"POLYGON ((135.76633 35.03832, 135.76657 35.037..."
3,261010000000004,26,101,000400000400,京都府,京都市北区,出雲路松ノ下町,None,5.279761e+04,1096.054,26,京都府,京都市北区,出雲路松ノ下町,000400000400,M,None,None,None,None,5,13,135.76300,35.04192,1010000000004,"POLYGON ((135.76154 35.04247, 135.76162 35.042..."
4,261010000000005,26,101,000500000500,京都府,京都市北区,大北山鏡石町,None,2.359074e+05,3045.946,26,京都府,京都市北区,大北山鏡石町,000500000500,M,None,None,None,None,3,20,135.73145,35.04665,1010000000005,"POLYGON ((135.73338 35.04831, 135.73332 35.047..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8284,264650000000016,26,465,002400002400,京都府,与謝郡与謝野町,字滝,None,1.557659e+07,24965.777,26,京都府,与謝郡与謝野町,字滝,002400002400,M,None,None,None,None,39,191,135.05751,35.46886,4650000000016,"POLYGON ((135.07700 35.48826, 135.07701 35.488..."
8285,264650000000017,26,465,002500002500,京都府,与謝郡与謝野町,字三河内,None,3.683443e+06,10826.044,26,京都府,与謝郡与謝野町,字三河内,002500002500,M,None,None,None,None,201,841,135.08837,35.51785,4650000000017,"POLYGON ((135.09067 35.52668, 135.09071 35.526..."
8286,264650000000018,26,465,002600002600,京都府,与謝郡与謝野町,字弓木,None,2.863092e+06,11378.146,26,京都府,与謝郡与謝野町,字弓木,002600002600,M,None,None,None,None,108,574,135.13991,35.55921,4650000000018,"POLYGON ((135.13857 35.56881, 135.13881 35.568..."
8287,264650000000019,26,465,002700002700,京都府,与謝郡与謝野町,字与謝,None,1.147165e+07,14232.546,26,京都府,与謝郡与謝野町,字与謝,002700002700,M,None,None,None,None,30,80,135.08430,35.45826,4650000000019,"POLYGON ((135.09537 35.47244, 135.09540 35.472..."


In [ ]:
for i in range(len(data)):
    if data.iloc[i, 0] is None:
        print(data.iloc[i])

KEY_CODE                                                   None
PREF                                                         26
CITY                                                        202
S_AREA                                                     None
PREF_NAME                                                  None
CITY_NAME                                                  None
S_NAME                                                     None
KIGO_E                                                     None
AREA                                                     651822
PERIMETER                                               3832.13
KEN                                                          26
KEN_NAME                                                   None
CSS_NAME                                                   None
MOJI                                                       None
AZA_CODE                                                   None
AREA_MAX_F                              

In [ ]:
# Noneの場所が特定できたので、整数型
data[data.index != 5677]['KEY_CODE'].astype(int)

0       261010000000001
1       261010000000002
2       261010000000003
3       261010000000004
4       261010000000005
             ...       
8284    264650000000016
8285    264650000000017
8286    264650000000018
8287    264650000000019
8288    264650000000020
Name: KEY_CODE, Length: 8288, dtype: int64

In [ ]:
data1 = data[data.index != 5677]
data1 = data1[['KEY_CODE', 'AREA', 'geometry']]
data1['KEY_CODE'] = data1['KEY_CODE'].astype(int)

In [ ]:
merged_data = data1.merge(df, on='KEY_CODE')

In [ ]:
merged_data

,KEY_CODE,AREA,geometry,CITY_NAME,AZA_CODE,AZA_NAME,総数（Ａ～Ｓ全産業）,Ａ～Ｒ全産業（Ｓ公務を除く）,Ａ～Ｂ農林漁業,Ｃ～Ｓ非農林漁業,Ｃ～Ｒ非農林漁業（Ｓ公務を除く）,Ｃ鉱業、採石業、砂利採取業,Ｄ建設業,Ｅ製造業,Ｆ電気・ガス・熱供給・水道業,Ｇ情報通信業,Ｈ運輸業、郵便業,Ｉ卸売業、小売業,Ｊ金融業、保険業,Ｋ不動産業、物品賃貸業,Ｌ学術研究、専門・技術サービス業,Ｍ宿泊業、飲食サービス業,Ｎ生活関連サービス業、娯楽業,Ｏ教育、学習支援業,Ｐ医療、福祉,Ｑ複合サービス事業,Ｒサービス業（他に分類されないもの）,Ｓ公務（他に分類されるものを除く）,１～４人,５～９人,１０～１９人,２０～２９人,３０人以上,出向・派遣従業者のみ,総数（Ａ～Ｓ全産業）_1,男（Ａ～Ｓ全産業）,女（Ａ～Ｓ全産業）,Ａ～Ｒ全産業（Ｓ公務を除く）_1,男Ａ～Ｒ全産業（Ｓ公務を除く）,女Ａ～Ｒ全産業（Ｓ公務を除く）,Ａ～Ｂ農林漁業_1,Ｃ～Ｓ非農林漁業_1,Ｃ～Ｒ非農林漁業（Ｓ公務を除く）_1,Ｃ鉱業、採石業、砂利採取業_1,Ｄ建設業_1,Ｅ製造業_1,Ｆ電気・ガス・熱供給・水道業_1,Ｇ情報通信業_1,Ｈ運輸業、郵便業_1,Ｉ卸売業、小売業_1,Ｊ金融業、保険業_1,Ｋ不動産業、物品賃貸業_1,Ｌ学術研究、専門・技術サービス業_1,Ｍ宿泊業、飲食サービス業_1,Ｎ生活関連サービス業、娯楽業_1,Ｏ教育、学習支援業_1,Ｐ医療、福祉_1,Ｑ複合サービス事業_1,Ｒサービス業（他に分類されないもの）_1,Ｓ公務（他に分類されるものを除く）_1,１～４人_1,５～９人_1,１０～１９人_1,２０～２９人_1,３０人以上_1,民営事業所数,うち個人,うち法人,うち会社,会社以外の法人,民営従業者数,うち個人_1,うち法人_1,うち会社_1,会社以外の法人_1,women_ratio
0,261010000000001,4.843208e+04,"POLYGON ((135.76674 35.03474, 135.76688 35.034...",京都市北区,1.000001e+08,出雲路神楽町,5,5,0,5,5,0,2,0,0,0,0,2,0,0,0,1,0,0,0,0,0,0,3,1,1,0,0,0,29,24,5,29,24,5,0,29,29,0,10,0,0,0,0,17,0,0,0,2,0,0,0,0,0,0,5,8,16,0,0,5,2,3,3,0,29,3,26,26,0,0.172414
1,261010000000002,6.134630e+04,"POLYGON ((135.76281 35.04064, 135.76292 35.040...",京都市北区,2.000002e+08,出雲路立テ本町,6,6,0,6,6,0,1,0,0,0,0,1,0,1,0,0,1,1,1,0,0,0,5,0,0,1,0,0,35,9,26,35,9,26,0,35,35,0,2,0,0,0,0,4,0,3,0,0,2,1,23,0,0,0,12,0,0,23,0,6,1,5,5,0,35,2,33,33,0,0.742857
2,261010000000003,6.526348e+04,"POLYGON ((135.76633 35.03832, 135.76657 35.037...",京都市北区,3.000003e+08,出雲路俵町,14,14,0,14,14,0,0,1,0,0,0,4,0,0,0,4,2,0,1,0,2,0,9,3,2,0,0,0,76,30,46,76,30,46,0,76,76,0,0,18,0,0,0,17,0,0,0,18,6,0,8,0,9,0,24,24,28,0,0,14,3,11,9,2,76,9,67,57,10,0.605263
3,261010000000004,5.279761e+04,"POLYGON ((135.76154 35.04247, 135.76162 35.042...",京都市北区,4.000004e+08,出雲路松ノ下町,5,5,0,5,5,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,2,0,4,1,0,0,0,0,13,6,7,13,6,7,0,13,13,0,0,0,0,0,0,0,0,1,0,5,0,0,3,0,4,0,8,5,0,0,0,5,2,3,2,1,13,4,9,6,3,0.538462
4,261010000000005,2.359074e+05,"POLYGON ((135.73338 35.04831, 135.73332 35.047...",京都市北区,5.000005e+08,大北山鏡石町,3,3,0,3,3,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,20,17,3,20,17,3,0,20,20,0,0,5,0,0,0,14,0,0,0,0,0,0,0,0,1,0,1,5,14,0,0,3,0,3,1,2,20,0,20,5,15,0.150000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8245,264650000000016,1.557659e+07,"POLYGON ((135.07700 35.48826, 135.07701 35.488...",与謝郡与謝野町,2.400002e+09,字滝,39,39,3,36,36,0,7,10,0,0,0,8,0,0,0,3,1,2,2,1,2,0,24,9,4,2,0,0,191,88,103,191,88,103,58,133,133,0,33,21,0,0,0,29,0,0,0,11,1,15,14,5,4,0,43,54,48,46,0,32,22,10,9,1,135,54,81,79,2,0.539267
8246,264650000000017,3.683443e+06,"POLYGON ((135.09067 35.52668, 135.09071 35.526...",与謝郡与謝野町,2.500002e+09,字三河内,201,201,0,201,201,0,20,93,0,0,1,33,2,2,6,7,10,11,10,2,4,0,162,22,11,2,4,0,841,406,435,841,406,435,0,841,841,0,87,233,0,0,2,192,6,10,23,17,40,95,93,10,33,0,309,152,162,45,173,196,149,47,40,7,751,326,425,337,88,0.517241
8247,264650000000018,2.863092e+06,"POLYGON ((135.13857 35.56881, 135.13881 35.568...",与謝郡与謝野町,2.600003e+09,字弓木,108,108,0,108,108,0,13,51,0,0,3,12,1,3,4,2,10,2,4,0,3,0,77,20,5,4,2,0,574,355,219,574,355,219,0,574,574,0,74,268,0,0,6,46,2,7,7,33,44,25,47,0,15,0,144,134,64,92,140,107,66,40,33,7,566,151,391,336,55,0.381533
8248,264650000000019,1.147165e+07,"POLYGON ((135.09537 35.47244, 135.09540 35.472...",与謝郡与謝野町,2.700003e+09,字与謝,30,30,2,28,28,0,1,22,0,0,0,3,0,0,0,1,0,0,0,0,1,0,27,2,1,0,0,0,80,50,30,80,50,30,10,70,70,0,13,46,0,0,0,8,0,0,0,2,0,0,0,0,1,0,54,13,13,0,0,29,24,5,4,1,78,47,31,30,1,0.375000


In [ ]:
merged_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 8250 entries, 0 to 8249
Data columns (total 76 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   KEY_CODE              8250 non-null   int64   
 1   AREA                  8250 non-null   float64 
 2   geometry              8250 non-null   geometry
 3   CITY_NAME             7103 non-null   object  
 4   AZA_CODE              7103 non-null   float64 
 5   AZA_NAME              7103 non-null   object  
 6   総数（Ａ～Ｓ全産業）            8250 non-null   int64   
 7   Ａ～Ｒ全産業（Ｓ公務を除く）        8250 non-null   int64   
 8   Ａ～Ｂ農林漁業               8250 non-null   int64   
 9   Ｃ～Ｓ非農林漁業              8250 non-null   int64   
 10  Ｃ～Ｒ非農林漁業（Ｓ公務を除く）      8250 non-null   int64   
 11  Ｃ鉱業、採石業、砂利採取業         8250 non-null   int64   
 12  Ｄ建設業                  8250 non-null   int64   
 13  Ｅ製造業                  8250 non-null   int64   
 14  Ｆ電気・ガス・熱供給・水道業        8250 non-null   int64   
 

In [ ]:
merged_data = merged_data.drop('women_ratio', axis=1)

In [ ]:
merged_data1 = merged_data.copy()


In [ ]:
merged_data1['person/area'] = merged_data1['総数（Ａ～Ｓ全産業）_1'] / merged_data1['AREA']
merged_data1['service/area'] = merged_data1['Ｍ宿泊業、飲食サービス業_1'] / merged_data1['AREA']
merged_data1['women_ratio'] = merged_data['女（Ａ～Ｓ全産業）'] / merged_data1['総数（Ａ～Ｓ全産業）_1']

In [ ]:
merged_data1[['person/area', 'service/area', 'women_ratio', 'Ｍ宿泊業、飲食サービス業_1']].describe([i/10 for i in range(0, 10)])

,person/area,service/area,women_ratio,Ｍ宿泊業、飲食サービス業_1
count,8250.000000,8250.000000,7101.000000,8250.000000
mean,0.062828,0.007160,0.454016,18.469455
std,1.213218,0.160245,0.181218,87.546097
min,0.000000,0.000000,0.000000,0.000000
0%,0.000000,0.000000,0.000000,0.000000
10%,0.000000,0.000000,0.217054,0.000000
20%,0.000017,0.000000,0.307692,0.000000
30%,0.000201,0.000000,0.374302,0.000000
40%,0.000682,0.000000,0.421053,0.000000
50%,0.001452,0.000000,0.465241,0.000000


In [ ]:
selected_data = merged_data1[(merged_data1['person/area'] > 0.015991) & (merged_data1['women_ratio'] > 0.67) & (merged_data1['service/area'] == 0) & (merged_data1["総数（Ａ～Ｓ全産業）_1"] > 146)]

In [ ]:
selected_data

,KEY_CODE,AREA,geometry,CITY_NAME,AZA_CODE,AZA_NAME,総数（Ａ～Ｓ全産業）,Ａ～Ｒ全産業（Ｓ公務を除く）,Ａ～Ｂ農林漁業,Ｃ～Ｓ非農林漁業,Ｃ～Ｒ非農林漁業（Ｓ公務を除く）,Ｃ鉱業、採石業、砂利採取業,Ｄ建設業,Ｅ製造業,Ｆ電気・ガス・熱供給・水道業,Ｇ情報通信業,Ｈ運輸業、郵便業,Ｉ卸売業、小売業,Ｊ金融業、保険業,Ｋ不動産業、物品賃貸業,Ｌ学術研究、専門・技術サービス業,Ｍ宿泊業、飲食サービス業,Ｎ生活関連サービス業、娯楽業,Ｏ教育、学習支援業,Ｐ医療、福祉,Ｑ複合サービス事業,Ｒサービス業（他に分類されないもの）,Ｓ公務（他に分類されるものを除く）,１～４人,５～９人,１０～１９人,２０～２９人,３０人以上,出向・派遣従業者のみ,総数（Ａ～Ｓ全産業）_1,男（Ａ～Ｓ全産業）,女（Ａ～Ｓ全産業）,Ａ～Ｒ全産業（Ｓ公務を除く）_1,男Ａ～Ｒ全産業（Ｓ公務を除く）,女Ａ～Ｒ全産業（Ｓ公務を除く）,Ａ～Ｂ農林漁業_1,Ｃ～Ｓ非農林漁業_1,Ｃ～Ｒ非農林漁業（Ｓ公務を除く）_1,Ｃ鉱業、採石業、砂利採取業_1,Ｄ建設業_1,Ｅ製造業_1,Ｆ電気・ガス・熱供給・水道業_1,Ｇ情報通信業_1,Ｈ運輸業、郵便業_1,Ｉ卸売業、小売業_1,Ｊ金融業、保険業_1,Ｋ不動産業、物品賃貸業_1,Ｌ学術研究、専門・技術サービス業_1,Ｍ宿泊業、飲食サービス業_1,Ｎ生活関連サービス業、娯楽業_1,Ｏ教育、学習支援業_1,Ｐ医療、福祉_1,Ｑ複合サービス事業_1,Ｒサービス業（他に分類されないもの）_1,Ｓ公務（他に分類されるものを除く）_1,１～４人_1,５～９人_1,１０～１９人_1,２０～２９人_1,３０人以上_1,民営事業所数,うち個人,うち法人,うち会社,会社以外の法人,民営従業者数,うち個人_1,うち法人_1,うち会社_1,会社以外の法人_1,person/area,service/area,women_ratio
597,261020000000149,1336.485,"POLYGON ((135.76573 35.02960, 135.76574 35.029...",京都市上京区,1.360001e+10,玄武町,11,11,0,11,11,0,0,0,0,0,0,2,0,0,1,0,0,5,0,0,3,0,2,1,1,1,6,0,2047,587,1460,2047,587,1460,0,2047,2047,0,0,0,0,0,0,310,0,0,5,0,0,1714,0,0,18,0,2,5,16,20,2004,11,0,10,0,10,2047,0,2046,0,2046,1.531630,0.0,0.713239
598,261020000000149,12615.209,"POLYGON ((135.76014 35.02970, 135.76031 35.029...",京都市上京区,1.360001e+10,玄武町,11,11,0,11,11,0,0,0,0,0,0,2,0,0,1,0,0,5,0,0,3,0,2,1,1,1,6,0,2047,587,1460,2047,587,1460,0,2047,2047,0,0,0,0,0,0,310,0,0,5,0,0,1714,0,0,18,0,2,5,16,20,2004,11,0,10,0,10,2047,0,2046,0,2046,0.162264,0.0,0.713239
599,261020000000149,17643.859,"POLYGON ((135.76439 35.02912, 135.76233 35.029...",京都市上京区,1.360001e+10,玄武町,11,11,0,11,11,0,0,0,0,0,0,2,0,0,1,0,0,5,0,0,3,0,2,1,1,1,6,0,2047,587,1460,2047,587,1460,0,2047,2047,0,0,0,0,0,0,310,0,0,5,0,0,1714,0,0,18,0,2,5,16,20,2004,11,0,10,0,10,2047,0,2046,0,2046,0.116018,0.0,0.713239
619,261020000000169,10611.755,"POLYGON ((135.75608 35.03151, 135.75609 35.031...",京都市上京区,1.530002e+10,近衛殿表町,4,4,0,4,4,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,3,0,288,86,202,288,86,202,0,288,288,0,0,35,0,0,0,144,0,0,2,0,0,107,0,0,0,0,2,0,0,0,286,4,0,4,1,3,288,0,288,35,253,0.027140,0.0,0.701389
1784,261040000000126,8418.417,"POLYGON ((135.75678 35.00490, 135.75678 35.004...",京都市中京区,1.100001e+10,小結棚町,20,20,0,20,20,0,0,1,0,0,1,8,2,3,3,0,0,0,0,0,2,0,5,6,2,2,5,0,1085,242,843,1085,242,843,0,1085,1085,0,0,3,0,0,6,723,284,15,11,0,0,0,0,0,43,0,11,40,27,47,960,20,1,19,17,2,1085,2,1083,1046,37,0.128884,0.0,0.776959
2133,261040000000468,10584.227,"POLYGON ((135.75495 35.00645, 135.75495 35.006...",京都市中京区,4.210004e+10,元本能寺南町,12,12,0,12,12,0,1,3,0,0,0,3,0,1,0,0,2,0,2,0,0,0,7,2,1,0,2,0,238,67,171,238,67,171,0,238,238,0,9,18,0,0,0,8,0,1,0,0,4,0,198,0,0,0,14,16,10,0,198,12,6,6,4,2,238,17,221,23,198,0.022486,0.0,0.718487
2923,261060000000478,6359.002,"POLYGON ((135.76490 34.99767, 135.76453 34.997...",京都市下京区,4.360004e+10,本上神明町,9,9,0,9,9,0,0,2,0,0,0,0,0,2,0,0,2,1,0,0,2,0,4,2,1,0,2,0,1171,365,806,1171,365,806,0,1171,1171,0,0,16,0,0,0,0,0,14,0,0,4,1,0,0,1136,0,6,14,15,0,1136,9,3,6,6,0,1171,5,1166,1166,0,0.184148,0.0,0.688301
3971,261090000000021,11361.640,"POLYGON ((135.75799 34.93837, 135.75764 34.938...",京都市伏見区,2.100002e+09,大宮町,12,12,0,12,12,0,0,1,0,0,0,6,1,0,0,0,2,1,1,0,0,0,8,2,1,0,1,0,332,37,295,332,37,295,0,332,332,0,0,1,0,0,0,17,19,0,0,0,3,5,287,0,0,0,14,12,19,0,287,12,8,4,2,2,332,18,314,8,306,0.029221,0.0,0.888554


In [ ]:
def make_folium_plot(data):
    center = shapely.geometry.MultiPolygon(data.geometry.values).centroid

    map = folium.Map([center.y, center.x])

    folium.Choropleth(geo_data=data, data=data, columns=['KEY_CODE', 'person/area'],
                    key_on='feature.properties.KEY_CODE', fill_color='OrRd', name='total').add_to(map)

    folium.GeoJson(
        data,
        tooltip=folium.features.GeoJsonTooltip(fields=['KEY_CODE', 'CITY_NAME', 'AZA_NAME', "総数（Ａ～Ｓ全産業）_1", "Ｍ宿泊業、飲食サービス業_1"]),
        style_function = lambda x: {'color': 'red'},
        name='under18',
    ).add_to(map)

    return map

In [ ]:
make_folium_plot(selected_data)

In [ ]:
selected_data2 = merged_data1[(merged_data1['person/area'] > 0.007) & (merged_data1['women_ratio'] > 0.6) & (merged_data1['service/area'] == 0) & (merged_data1["総数（Ａ～Ｓ全産業）_1"] > 146)]

In [ ]:
make_folium_plot(selected_data2)

In [ ]:
selected_data3 = merged_data1[(merged_data1['person/area'] > 0.007) & (merged_data1['women_ratio'] > 0.6) & (merged_data1['Ｍ宿泊業、飲食サービス業_1'] > 41) & (merged_data1["総数（Ａ～Ｓ全産業）_1"] > 146)]
len(selected_data3)

86

In [ ]:
make_folium_plot(selected_data3)